In [1]:
import pandas as pd
from tqdm.notebook import tqdm
import time
from datetime import datetime, timedelta
from topaz import TopazAPI
from sklearn.preprocessing import MinMaxScaler 
import numpy as np
import requests
import os
import concurrent.futures
import math
api_key = '51066aa1-85af-4f6f-aaa5-08b0f3133af5' #Insert your API key 
topaz_api = TopazAPI(api_key)
import mapping


In [6]:
import betfairlightweight
from betfairlightweight import filters
import pandas as pd
from datetime import timedelta
from nltk.tokenize import regexp_tokenize
import warnings
import json
# data_tools.ft_sec_key import SECKEY

warnings.filterwarnings('ignore', message='The behavior of DataFrame concatenation with empty or all-NA entries is deprecated.*')

def bflw_trading():
    '''
    This function loads the credentials file, and passes the credentials into the betfairlightweight instance
    '''

    username = 'nickbarlow@live.com.au'
    password = 'un6/chxe!N!?adsp'
    app_key = 'JFWqJHqB4Akfi5hK'

    # Define the betfairlightweight client
    trading = betfairlightweight.APIClient(username, password, app_key=app_key)

    return trading

def login(trading):
    # login to the API
    trading.login_interactive()

def greyhound_market_filter():
    # Define the greyhound market filter
    market_filter = filters.market_filter(
        event_type_ids=[4339],  # For horse racing
        market_countries=['AU'],  # For Australia
        market_type_codes=['WIN']  # For win markets
    )

    return market_filter

def process_runner_books(runner_books):
    # Define the fields required from the runner book
    selection_ids = [runner_book.selection_id for runner_book in runner_books]

    df = pd.DataFrame({
        'selectionId': selection_ids,
    })
    return df

def generate_greyhound_catalogue(trading,market_filter):
    # Load the greyhound market catalogues from the Betfair API
    greyhound_market_catalogues = trading.betting.list_market_catalogue(
    filter=market_filter,
    market_projection=['RUNNER_DESCRIPTION', 'EVENT', 'MARKET_DESCRIPTION'],
    max_results='200')

    print(f"Found {len(greyhound_market_catalogues)} markets.")

    return greyhound_market_catalogues

RUNNER_DATA_COLUMNS = [
            'marketStart',
            'track',
            'raceNumber',
            'raceType',
            'winMarketId',
            'selectionId',
            'rugNumber',
            'boxNumber',
            'dogName'
            ]

def initilise_dataframe():
    # Create the empty dataframe
    data = pd.DataFrame(columns=RUNNER_DATA_COLUMNS)

    return data

PATTERN1 = r'(?<=<br>Dog ).+?(?= starts)'

PATTERN2 = r"(?<=\bbox no. )(\w+)"

def process_market_clarifications(runners_df,clarifications):
    '''
    This function accesses the market clarifications field which explains which box the reserve runner will be starting from (if any) and parses the information using regex
    We utilise this information rather than the Topaz API data because Betfair markets only use final field information

    A clarification will look like: "<br>Box changes:<br>Dog 9. Tralee Blaze starts from box no. 8<br><br>Dog 6. That Other One starts from box no. 2<br><br>"
    '''
    # Define the clarifications dataframe
    market_clarifications = pd.DataFrame(regexp_tokenize(clarifications, PATTERN1), columns = ['dogName'])

    # Remove dog name from runner_number
    market_clarifications['rugNumber'] = market_clarifications['dogName'].str.split(r'. ').str[0]

    # Extract box number from clarifications
    market_clarifications['boxNumber'] = regexp_tokenize(clarifications, PATTERN2)

    # Keep only boxNumber and rugNumber
    market_clarifications=market_clarifications[['rugNumber','boxNumber']]

    # Merge the clarifications with the original dataframe
    runners_df = pd.merge(runners_df,market_clarifications,how='left',on=['rugNumber'])

    # Any runners with no clarifications will start in the box that matches the rugNumber
    runners_df['boxNumber'].fillna(runners_df['rugNumber'],inplace=True)

    return runners_df


def collect_greyhound_market_data(trading,greyhound_market_catalogues,data):
    '''
    This function will process the greyhound market catalogue to access information about the market including:
     - Market ID
     - Market Name
     - Event Name
     - Start Time
     - Clarifications

    It will then process each individual market book to gather the runner information, following by some operations to put market information into the dataframe columns including adjusting the timezone from UTC to AEST
    Finally it will then perform some string splitting operations to generate more useful market/runner information:
     - Track
     - Race Number
     - Race Type
     - Rug Number
     - Dog Name

    These operations may be useful depending on whether the betting intention is for a specific subset of races. It is also possible to split out race distance from the market name
    '''
    # Initiate the for loop
    for market_catalogue in greyhound_market_catalogues:

        # Name variables for market parameters
        market_id = market_catalogue.market_id
        market_name = market_catalogue.market_name
        event_name = market_catalogue.event.name
        market_start_time = market_catalogue.description.market_time

        # Try to access clarifications and replace a known string replacement to prepare it for our regex functuon
        try:
            clarifications = market_catalogue.description.clarifications.replace("<br> Dog","<br>Dog")
        except AttributeError:
            clarifications = None

        # Generate our market_books list
        market_books = trading.betting.list_market_book(market_ids=[market_id])

        # Generate our runner_catalogues list
        runner_catalogues = market_catalogue.runners

        # Initiate the market_books for loop
        for market_book in market_books:

            # Call the process_runner_books function
            runners_df = process_runner_books(market_book.runners)

            # Get the runner catalogue
            for runner in market_book.runners:

                # define the runner catalogue
                runner_catalogue = next((rd for rd in runner_catalogues if rd.selection_id == runner.selection_id), None)

                # define the runner name for non-empty runner_catalogues
                if runner_catalogue is not None:
                    runner_name = runner_catalogue.runner_name
                    runners_df.loc[runners_df['selectionId'] == runner.selection_id, 'dogName'] = runner_name

            # Assign market variables to the dataframe
            runners_df['winMarketId'] = market_id
            runners_df['marketName'] = market_name
            runners_df['eventName'] = event_name
            runners_df['marketStart'] = market_start_time

            # Adjust the timezone from UTC to AEST
            runners_df['marketStart'] = runners_df['marketStart'] + timedelta(hours=10)

            # Perform string split operations 
            runners_df['track']=runners_df['eventName'].str.split(' \(').str[0]
            runners_df['raceNumber']=runners_df['marketName'].str.split(r' ').str[0]
            runners_df['raceNumber']=runners_df['raceNumber'].str.split('R').str[1]
            runners_df['raceType']=runners_df['marketName'].str.split(r'm ').str[1]
            runners_df['rugNumber']=runners_df['dogName'].str.split(r'. ').str[0]
            runners_df['dogName']=runners_df['dogName'].str.split('\. ').str[1]

            # Call the process_market_clarifications function. If there no reserve runners running then the boxNumber = rugNumber
            try:
                runners_df = process_market_clarifications(runners_df,clarifications)
            except TypeError:
                runners_df['boxNumber'] = runners_df['rugNumber']

            # concatenate the dataframes together
            data=pd.concat([data,runners_df], sort=False)

    # Keep only required columns
    data = data[RUNNER_DATA_COLUMNS]
    data = pd.DataFrame(data)

    print(data.head)

    return data

def download_betfair_market_data():
    '''
    This function combines all our previously defined functions to generate our market csv from the Betfair API
    '''
    trading = bflw_trading()

    login(trading)

    market_filter = greyhound_market_filter()

    greyhound_market_catalogues = generate_greyhound_catalogue(trading,market_filter)

    data = initilise_dataframe()

    data = collect_greyhound_market_data(trading,greyhound_market_catalogues,data)

    return data

betfair_data = download_betfair_market_data()

# def upcoming_topaz_data(codes,datatype,betfair_data):
#     '''
#     This function loads our upcoming races, discards the Topaz API boxNumber and adds the boxNumber information retrieved from the Betfair API
#     '''
#     # Load today's race information
#     TodaysTopazData = load_topaz_data(codes,datatype)

#     # Keep only required Betfair information
#     betfair_fields = betfair_data[['track','raceNumber','rugNumber','boxNumber']]

#     # Discard the Topaz API boxNumber information
#     TodaysTopazData.drop(columns=['boxNumber'], inplace=True)

#     # Merge the Betfair boxNumber information
#     TodaysTopazData = pd.merge(TodaysTopazData,betfair_fields,how='left',on=['track','raceNumber','rugNumber'])

#     return TodaysTopazData

# TodaysTopazData = (JURISDICTION_CODES,'UPCOMING',betfair_data)

# def concatenate_data(TopazDataHistorical,TodaysTopazData):

#     # Concatenate the last 12 months of Topaz Data with today's races
#     TopazDataPreProcessing = pd.concat([TopazDataHistorical,TodaysTopazData])

#     return TopazDataPreProcessing

# TopazDataPreProcessing = concatenate_data(TopazDataHistorical,TodaysTopazData)

Found 3 markets.
<bound method NDFrame.head of           marketStart     track raceNumber raceType  winMarketId selectionId  \
0 2024-05-09 23:53:00  Mandurah         10      Gr6  1.228672956    49766619   
1 2024-05-09 23:53:00  Mandurah         10      Gr6  1.228672956    52546019   
2 2024-05-09 23:53:00  Mandurah         10      Gr6  1.228672956    69129000   
3 2024-05-09 23:53:00  Mandurah         10      Gr6  1.228672956    58280358   
4 2024-05-09 23:53:00  Mandurah         10      Gr6  1.228672956    52311116   
5 2024-05-09 23:53:00  Mandurah         10      Gr6  1.228672956    64052135   
6 2024-05-09 23:53:00  Mandurah         10      Gr6  1.228672956    65293021   
7 2024-05-09 23:53:00  Mandurah         10      Gr6  1.228672956    47727880   
0 2024-05-10 00:15:00  Mandurah         11      Gr6  1.228672961    49009152   
1 2024-05-10 00:15:00  Mandurah         11      Gr6  1.228672961    42401359   
2 2024-05-10 00:15:00  Mandurah         11      Gr6  1.228672961    63609

In [7]:
betfair_data

,marketStart,track,raceNumber,raceType,winMarketId,selectionId,rugNumber,boxNumber,dogName
0,2024-05-09 23:53:00,Mandurah,10,Gr6,1.228672956,49766619,1,1,Hello Dave
1,2024-05-09 23:53:00,Mandurah,10,Gr6,1.228672956,52546019,2,2,Bad Influence
2,2024-05-09 23:53:00,Mandurah,10,Gr6,1.228672956,69129000,3,3,West On Marshall
3,2024-05-09 23:53:00,Mandurah,10,Gr6,1.228672956,58280358,4,4,Royal Fun
4,2024-05-09 23:53:00,Mandurah,10,Gr6,1.228672956,52311116,5,5,Lenny Coaster
5,2024-05-09 23:53:00,Mandurah,10,Gr6,1.228672956,64052135,6,6,Uncompromised
6,2024-05-09 23:53:00,Mandurah,10,Gr6,1.228672956,65293021,7,7,West On Libby
7,2024-05-09 23:53:00,Mandurah,10,Gr6,1.228672956,47727880,8,8,Abigail Gundi
0,2024-05-10 00:15:00,Mandurah,11,Gr6,1.228672961,49009152,1,1,Poppy Tahleia
1,2024-05-10 00:15:00,Mandurah,11,Gr6,1.228672961,42401359,2,2,Blazing Joy


In [2]:
runs_df = pd.concat([pd.read_feather('server_data/'+x) for x in os.listdir('server_data/')])
runs_df = runs_df.dropna(subset=['place'], how='all')    
runs_df.drop_duplicates(inplace=True)
runs_df['date'] = pd.to_datetime(runs_df['meetingDate']).dt.date
runs_df['year-month'] = pd.to_datetime(runs_df['meetingDate']).dt.to_period('M')
runs_df['year'] = pd.to_datetime(runs_df['meetingDate']).dt.to_period('Y')

C:\Users\Nick\AppData\Local\Temp\ipykernel_24192\2211342402.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  runs_df = pd.concat([pd.read_feather('server_data/'+x) for x in os.listdir('server_data/')])
C:\Users\Nick\AppData\Local\Temp\ipykernel_24192\2211342402.py:5: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  runs_df['year-month'] = pd.to_datetime(runs_df['meetingDate']).dt.to_period('M')
C:\Users\Nick\AppData\Local\Temp\ipykernel_24192\2211342402.py:6: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  runs_df['year'] = pd.to_datetime(runs_df['meetingDate']).dt.to_period('Y')


In [3]:
TrackDict = {
    'Auckland (NZ)':'Manukau',
    'Christchurch (NZ)':'Addington',
    'Dport @ HOB':'Hobart',
    'Dport @ LCN':'Launceston',
    'Meadows (MEP)':'The Meadows',
    'Otago (NZ)':'Forbury Park',
    'Palmerston Nth (NZ)':'Manawatu',
    'Sandown (SAP)':'Sandown Park',
    'Southland (NZ)':'Ascot Park',
    'Tokoroa (NZ)':'Tokoroa',
    'Waikato (NZ)':'Cambridge',
    'Wanganui (NZ)':'Hatrick',
    'Taranaki (NZ)':'Taranaki',
    'Ashburton (NZ)':'Ashburton',
    'Richmond (RIS)':'Richmond Straight',
    'Murray Bridge (MBR)':'Murray Bridge',
    'Murray Bridge (MBS)':'Murray Bridge Straight'
}
TopazData = runs_df.copy()
TopazData['track'] = TopazData['track'].replace(TrackDict)
TopazData['meetingDate'] = pd.to_datetime(TopazData['meetingDate'])
TopazData['dateWhelped'] = pd.to_datetime(TopazData['dateWhelped'])

TopazData['dogName']=TopazData['dogName'].str.replace("'","")
TopazData['sireName']=TopazData['sireName'].str.replace("'","")
TopazData['damName']=TopazData['damName'].str.replace("'","")
state_map = {x['trackName']:x['State'] for x in mapping.trackCodes}
TopazData['state'] = TopazData['track'].map(state_map)

In [4]:
TopazData = TopazData.query('state == "VIC"')

In [5]:
TopazData.date.max()

datetime.date(2024, 4, 26)

In [6]:
start_date = datetime(2024,4,27)

In [7]:
new_races = topaz_api.get_races(start_date)
new_races

,raceId,meetingId,raceNumber,raceTypeCode,raceType,name,startTime,raceStart,distance,isBoxDrawn,...,tip3,tip4,winPool,placePool,trifectaPool,owningAuthorityCode,isRaceResultEntered,photoFinishUrl,resultYouTubeVideoId,tabTipRunners
0,1024571056,900972885,1,M,Maiden,NATIONAL GREYHOUND ADOPTION MONTH,7:08PM,2024-04-26T09:08:00.000Z,400,True,...,8,6,8298.72,2992.90,5702.49,VIC,True,https://fasttrack.blob.core.windows.net/webcon...,bekCGDYZK4g,"[3, 1, 8, 6]"
1,1024571062,900972885,2,M,Maiden,THE BECKLEY CENTRE GEELONG,7:31PM,2024-04-26T09:31:00.000Z,460,True,...,4,2,8544.15,2538.33,4533.76,VIC,True,https://fasttrack.blob.core.windows.net/webcon...,EOQl8_MV-AQ,"[7, 8, 4, 2]"
2,1022172278,900972885,3,MF,Maiden Final,SPORTSBET FINAL,7:49PM,2024-04-26T09:49:00.000Z,460,True,...,8,2,7924.18,2947.99,5840.28,VIC,True,https://fasttrack.blob.core.windows.net/webcon...,r9vZxoQRAjc,"[3, 6, 8, 2]"
3,1024571061,900972885,4,X67,Mixed 6/7,MERV JENNINGS SIGNS,8:08PM,2024-04-26T10:08:00.000Z,460,True,...,6,7,5505.53,2255.14,4029.45,VIC,True,https://fasttrack.blob.core.windows.net/webcon...,FAE2CGcRPpo,"[8, 5, 6, 7]"
4,1024571060,900972885,5,5,Grade 5,MILLERS MUZZLES,8:31PM,2024-04-26T10:31:00.000Z,460,True,...,7,8,7840.49,2114.91,5212.14,VIC,True,https://fasttrack.blob.core.windows.net/webcon...,UicbD2-IMkM,"[6, 3, 7, 8]"
5,1024571058,900972885,6,5,Grade 5,TTS ELECTRICS,8:49PM,2024-04-26T10:49:00.000Z,460,True,...,2,6,7887.79,3627.79,6427.94,VIC,True,https://fasttrack.blob.core.windows.net/webcon...,BzsUB-rgv0U,"[8, 4, 2, 6]"
6,1024571059,900972885,7,5,Grade 5,PARKER PUMPS,9:08PM,2024-04-26T11:08:00.000Z,460,True,...,3,5,7176.18,2632.10,4592.65,VIC,True,https://fasttrack.blob.core.windows.net/webcon...,e7c-wrkaINA,"[1, 2, 3, 5]"
7,1024571057,900972885,8,X34,Mixed 3/4,TOPCAT VIDEO PRODUCTIONS,9:32PM,2024-04-26T11:32:00.000Z,460,True,...,6,1,5855.88,1983.41,4344.34,VIC,True,https://fasttrack.blob.core.windows.net/webcon...,zNPlZfZP4y4,"[7, 8, 6, 1]"
8,1024571053,900972885,9,X67,Mixed 6/7,DOWNLOAD THE WATCHDOG APP,9:54PM,2024-04-26T11:54:00.000Z,400,True,...,6,1,5291.75,1893.88,3952.85,VIC,True,https://fasttrack.blob.core.windows.net/webcon...,_RL-AzvMQCw,"[4, 8, 6, 1]"
9,1024571052,900972885,10,6G,Grade 6,NP ELECTRICAL,10:15PM,2024-04-26T12:15:00.000Z,400,True,...,2,7,6964.03,2352.39,5349.96,VIC,True,https://fasttrack.blob.core.windows.net/webcon...,lAndjaqO3HE,"[6, 3, 2, 7]"


In [8]:
new_races.columns

Index(['raceId', 'meetingId', 'raceNumber', 'raceTypeCode', 'raceType', 'name',
       'startTime', 'raceStart', 'distance', 'isBoxDrawn', 'prizeMoney1',
       'prizeMoney2', 'prizeMoney3', 'prizeMoney4', 'prizeMoney5',
       'prizeMoney6', 'prizeMoney7', 'prizeMoney8', 'prizeMoneyTotal',
       'analystComment', 'suggestedBetTypeCode', 'suggestedBet',
       'isDailyDouble', 'isQuaddie', 'isEarlyQuaddie', 'isRunningDouble',
       'isTrifecta', 'isQuinella', 'isExacta', 'isPickFour', 'isBixSix',
       'tip1', 'tip2', 'tip3', 'tip4', 'winPool', 'placePool', 'trifectaPool',
       'owningAuthorityCode', 'isRaceResultEntered', 'photoFinishUrl',
       'resultYouTubeVideoId', 'tabTipRunners'],
      dtype='object')

In [9]:
new_race_ids = new_races['raceId'].unique()
new_race_ids

array([1024571056, 1024571062, 1022172278, 1024571061, 1024571060,
       1024571058, 1024571059, 1024571057, 1024571053, 1024571052,
       1024571054, 1024571055, 1024028484, 1024028485, 1024028483,
       1024028481, 1024028488, 1024028491, 1022226403, 1024028490,
       1024028482, 1024028486, 1024028487, 1024028489, 1024571071,
       1024571064, 1024571063, 1024571066, 1024571074, 1024571072,
       1024571073, 1024571065, 1024571067, 1024571068, 1024571069,
       1024571070, 1024620661, 1024620660, 1024620659, 1024620658,
       1024620652, 1024620651, 1024620653, 1024620662, 1024620657,
       1024620656, 1024620654, 1024620655], dtype=int64)

In [10]:
race_run = topaz_api.get_race_runs(race_id = 1029602600)

In [11]:
race_run['meetingDate'] = pd.to_datetime(race_run['meetingDate'])

In [12]:
race_run

,trackCode,track,distance,raceId,meetingDate,raceTypeCode,raceType,runId,dogId,dogName,...,sireName,dateWhelped,totalFormCount,last5,isLateScratching,bestTime,bestFinishTrackAndDistance,pir,careerPrizeMoney,averageSpeed
0,GEL,Geelong,400,1029602600,2024-05-07 00:00:00+00:00,M,Maiden,1029603145,679825474,LITERALLY,...,DELIVER,2021-03-14T00:00:00.000Z,0,2-4-5-3-5,False,NBT,23.010,None,1250,61.986000
1,GEL,Geelong,400,1029602600,2024-05-07 00:00:00+00:00,M,Maiden,1029603146,829185569,CHARLI MARLEY,...,SH AVATAR,2022-07-02T00:00:00.000Z,0,6-4-4-3,False,NBT,23.327,None,385,64.375000
2,GEL,Geelong,400,1029602600,2024-05-07 00:00:00+00:00,M,Maiden,1029603148,829185567,NAILED IT,...,SH AVATAR,2022-07-02T00:00:00.000Z,0,7-7-5-6,False,FSH,NaN,None,0,61.322500
3,GEL,Geelong,400,1029602600,2024-05-07 00:00:00+00:00,M,Maiden,1029603149,810008286,ACTION ROYALE,...,KEYBOW,2022-04-10T00:00:00.000Z,0,4-7-3-3-7,False,NBT,23.173,None,1210,63.682000
4,GEL,Geelong,400,1029602600,2024-05-07 00:00:00+00:00,M,Maiden,1029603147,709150473,JEFF THE MAN,...,MY REDEEMER,2021-05-09T00:00:00.000Z,0,4-4-2-3-2,False,NBT,22.987,None,1400,61.788000
5,GEL,Geelong,400,1029602600,2024-05-07 00:00:00+00:00,M,Maiden,1029603144,832005697,SUDOKU NALA,...,FERNANDO BALE,2022-07-01T00:00:00.000Z,0,8-3,False,FSH,NaN,None,260,61.935000
6,GEL,Geelong,400,1029602600,2024-05-07 00:00:00+00:00,M,Maiden,1029603143,801425181,JAIRON BALE,...,FERNANDO BALE,2022-03-18T00:00:00.000Z,0,6-5-2-7,False,NBT,22.978,None,320,62.072500
7,GEL,Geelong,400,1029602600,2024-05-07 00:00:00+00:00,M,Maiden,1029603142,713187529,BROWSER,...,HOOKED ON SCOTCH,2021-05-08T00:00:00.000Z,0,5-3-2-4-2,False,NBT,23.099,None,1055,65.938571
8,GEL,Geelong,400,1029602600,2024-05-07 00:00:00+00:00,M,Maiden,1029603140,791442231,SWEET GIZMO,...,GET IT GIZMO,2022-02-10T00:00:00.000Z,0,6-8-8-8-5,False,NBT,23.899,None,0,61.731429
9,GEL,Geelong,400,1029602600,2024-05-07 00:00:00+00:00,M,Maiden,1029603141,785329025,MORAINE ANNIE,...,BEAST UNLEASHED,2022-02-08T00:00:00.000Z,0,7-5-8-8-6,False,NBT,23.835,None,0,59.895000


In [13]:
TopazData.columns

Index(['trackCode', 'track', 'distance', 'raceId', 'meetingDate',
       'raceTypeCode', 'raceType', 'runId', 'dogId', 'dogName', 'weightInKg',
       'incomingGrade', 'outgoingGrade', 'gradedTo', 'rating', 'raceNumber',
       'boxNumber', 'boxDrawnOrder', 'rugNumber', 'startPrice', 'place',
       'unplaced', 'unplacedCode', 'scratched', 'prizeMoney', 'resultTime',
       'resultMargin', 'resultMarginLengths', 'startPaceCode', 'jumpCode',
       'runLineCode', 'firstSecond', 'colourCode', 'sex', 'comment', 'ownerId',
       'trainerId', 'ownerName', 'ownerState', 'trainerName', 'trainerSuburb',
       'trainerState', 'trainerDistrict', 'trainerPostCode', 'isQuad',
       'isBestBet', 'damId', 'damName', 'sireId', 'sireName', 'dateWhelped',
       'totalFormCount', 'last5', 'isLateScratching', 'bestTime',
       'bestFinishTrackAndDistance', 'pir', 'careerPrizeMoney', 'averageSpeed',
       'time_1', 'position_1', 'splitMargin_1', 'time_2', 'position_2',
       'splitMargin_2', 'date'

In [14]:
new_w_form = pd.concat([TopazData,race_run])

C:\Users\Nick\AppData\Local\Temp\ipykernel_24192\1883169934.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_w_form = pd.concat([TopazData,race_run])


In [15]:
new_w_form

,trackCode,track,distance,raceId,meetingDate,raceTypeCode,raceType,runId,dogId,dogName,...,time_1,position_1,splitMargin_1,time_2,position_2,splitMargin_2,date,year-month,year,state
1043,GEL,Geelong,400,861582735,2023-01-06 00:00:00+00:00,M,Maiden,861583304,680626042,RUSHMORE RED,...,8.6175,3.0,0.0700,NaN,NaN,None,2023-01-06,2023-01,2023,VIC
1044,GEL,Geelong,400,861582735,2023-01-06 00:00:00+00:00,M,Maiden,861583299,715726098,USAIN ROAD,...,8.8625,6.0,0.1575,NaN,NaN,None,2023-01-06,2023-01,2023,VIC
1045,GEL,Geelong,400,861582735,2023-01-06 00:00:00+00:00,M,Maiden,861583305,700541794,DASH OF SALLY,...,8.6700,4.0,0.0525,NaN,NaN,None,2023-01-06,2023-01,2023,VIC
1046,GEL,Geelong,400,861582735,2023-01-06 00:00:00+00:00,M,Maiden,861583300,632134753,MAGNETIC FORCE,...,8.9850,7.0,0.1225,NaN,NaN,None,2023-01-06,2023-01,2023,VIC
1047,GEL,Geelong,400,861582735,2023-01-06 00:00:00+00:00,M,Maiden,861583306,578790276,LEXI THE STAR,...,9.0025,8.0,0.0175,NaN,NaN,None,2023-01-06,2023-01,2023,VIC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,GEL,Geelong,400,1029602600,2024-05-07 00:00:00+00:00,M,Maiden,1029603144,832005697,SUDOKU NALA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN
6,GEL,Geelong,400,1029602600,2024-05-07 00:00:00+00:00,M,Maiden,1029603143,801425181,JAIRON BALE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN
7,GEL,Geelong,400,1029602600,2024-05-07 00:00:00+00:00,M,Maiden,1029603142,713187529,BROWSER,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN
8,GEL,Geelong,400,1029602600,2024-05-07 00:00:00+00:00,M,Maiden,1029603140,791442231,SWEET GIZMO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN


In [43]:
TopazData = Topazbackup

In [16]:
Topazbackup = TopazData.copy()


In [18]:
TopazData = new_w_form.copy()

In [19]:
TopazData

,trackCode,track,distance,raceId,meetingDate,raceTypeCode,raceType,runId,dogId,dogName,...,time_1,position_1,splitMargin_1,time_2,position_2,splitMargin_2,date,year-month,year,state
1043,GEL,Geelong,400,861582735,2023-01-06 00:00:00+00:00,M,Maiden,861583304,680626042,RUSHMORE RED,...,8.6175,3.0,0.0700,NaN,NaN,None,2023-01-06,2023-01,2023,VIC
1044,GEL,Geelong,400,861582735,2023-01-06 00:00:00+00:00,M,Maiden,861583299,715726098,USAIN ROAD,...,8.8625,6.0,0.1575,NaN,NaN,None,2023-01-06,2023-01,2023,VIC
1045,GEL,Geelong,400,861582735,2023-01-06 00:00:00+00:00,M,Maiden,861583305,700541794,DASH OF SALLY,...,8.6700,4.0,0.0525,NaN,NaN,None,2023-01-06,2023-01,2023,VIC
1046,GEL,Geelong,400,861582735,2023-01-06 00:00:00+00:00,M,Maiden,861583300,632134753,MAGNETIC FORCE,...,8.9850,7.0,0.1225,NaN,NaN,None,2023-01-06,2023-01,2023,VIC
1047,GEL,Geelong,400,861582735,2023-01-06 00:00:00+00:00,M,Maiden,861583306,578790276,LEXI THE STAR,...,9.0025,8.0,0.0175,NaN,NaN,None,2023-01-06,2023-01,2023,VIC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,GEL,Geelong,400,1029602600,2024-05-07 00:00:00+00:00,M,Maiden,1029603144,832005697,SUDOKU NALA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN
6,GEL,Geelong,400,1029602600,2024-05-07 00:00:00+00:00,M,Maiden,1029603143,801425181,JAIRON BALE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN
7,GEL,Geelong,400,1029602600,2024-05-07 00:00:00+00:00,M,Maiden,1029603142,713187529,BROWSER,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN
8,GEL,Geelong,400,1029602600,2024-05-07 00:00:00+00:00,M,Maiden,1029603140,791442231,SWEET GIZMO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN


In [44]:
# TopazData['last5'] = TopazData['last5'].astype(str)
scaler = MinMaxScaler()
TopazData['track'] = TopazData['track'].replace(TrackDict)
TopazData['meetingDate'] = pd.to_datetime(TopazData['meetingDate'])
TopazData['dateWhelped'] = pd.to_datetime(TopazData['dateWhelped'])

TopazData['dogName']=TopazData['dogName'].str.replace("'","")
TopazData['sireName']=TopazData['sireName'].str.replace("'","")
TopazData['damName']=TopazData['damName'].str.replace("'","")
# # Convert the 'pir' column to string
try:
    TopazData['pir'] = TopazData['pir'].fillna(0)
    TopazData['pir'] = TopazData['pir'].astype(int).astype(str)
except ValueError:
    print('Error converting pir to string')
    TopazData['pir'] = '000'

# # Extract the second last letter and create a new column '2ndLastPIR'
TopazData['2ndLastPIR'] = TopazData['pir'].apply(lambda x: x[-2] if len(x) >= 2 else None)
TopazData['2ndLastPIR'] = TopazData['2ndLastPIR'].fillna(TopazData['place']).fillna(0)
TopazData['2ndLastPIR'] = TopazData['2ndLastPIR'].astype(int)

# # Create a feature that calculates places gained/conceded in the home straight
TopazData['finishingPlaceMovement'] = TopazData['2ndLastPIR'] - TopazData['place']

TopazData['weightInKgScaled'] = TopazData.groupby('raceId')['weightInKg'].transform(lambda x: scaler.fit_transform(x.values.reshape(-1, 1)).flatten() if x.nunique() > 1 else 0)
# TopazData['weightInKgScaled'] = TopazData.groupby('raceId')['weightInKg'].transform(lambda x: scaler.fit_transform(x.values.reshape(-1, 1)).flatten() if x.nunique() > 1 else 0)
#Scale values as required
TopazData['prizemoneyLog'] = np.log10(TopazData['prizeMoney'] + 1)
TopazData['placeLog'] = np.log10(TopazData['place'] + 1)
TopazData['marginLog'] = np.log10(TopazData['resultMargin'] + 1)

# Calculate median winner time per track/distance
win_results = TopazData[TopazData['place'] == 1]

grouped_data = win_results.groupby(['track', 'distance', 'meetingDate'])['resultTime'].median().reset_index()

median_win_time = pd.DataFrame(grouped_data.groupby(['track', 'distance']).apply(lambda x: x.sort_values('meetingDate').set_index('meetingDate')['resultTime'].shift(1).rolling('365D', min_periods=1).median())).reset_index()
median_win_time.rename(columns={"resultTime": "runTimeMedian"},inplace=True)

median_win_time['speedIndex'] = (median_win_time['runTimeMedian'] / median_win_time['distance'])
median_win_time['speedIndex'] = MinMaxScaler().fit_transform(median_win_time[['speedIndex']])

TopazData = TopazData.merge(median_win_time, how='left', on=['track', 'distance','meetingDate'])

TopazData['runTimeNorm'] = (TopazData['runTimeMedian'] / TopazData['resultTime']).clip(0.8, 1.2)
TopazData['runTimeNorm'] = MinMaxScaler().fit_transform(TopazData[['runTimeNorm']])

#Same for split time
split_win_results = TopazData[TopazData['position_1'] == 1]
grouped_data = split_win_results.groupby(['track', 'distance', 'meetingDate'])['time_1'].median().reset_index()

median_split_win_time = pd.DataFrame(grouped_data.groupby(['track', 'distance']).apply(lambda x: x.sort_values('meetingDate').set_index('meetingDate')['time_1'].shift(1).rolling('365D', min_periods=1).median())).reset_index()
median_split_win_time.rename(columns={'time_1': 'split_time_median'},inplace=True)

median_split_win_time['split_speedIndex'] = median_split_win_time['split_time_median']
median_split_win_time['split_speedIndex'] = MinMaxScaler().fit_transform(median_split_win_time[['split_time_median']])

# Merge with median winner time
TopazData = TopazData.merge(median_split_win_time, how='left', on=['track', 'distance','meetingDate'])

# Normalise time comparison
TopazData['split_runTimeNorm'] = (TopazData['split_time_median'] / TopazData['time_1']).clip(0.5, 1.5)
TopazData['split_runTimeNorm'] = MinMaxScaler().fit_transform(TopazData[['split_runTimeNorm']])

min_run_time = TopazData.groupby('raceId')[['time_1']].min().reset_index().rename(columns={'time_1':'min_run_time'})
TopazData = TopazData.merge(min_run_time, on='raceId')
TopazData['split_time_margin'] = TopazData['time_1']-TopazData['min_run_time']
# Same for runhome time
TopazData['run_home_time'] = TopazData['resultTime'] - TopazData['time_1']
win_results = TopazData[TopazData['place'] == 1]


grouped_data = win_results.groupby(['track', 'distance', 'meetingDate'])['run_home_time'].median().reset_index()

median_win_time = pd.DataFrame(grouped_data.groupby(['track', 'distance']).apply(lambda x: x.sort_values('meetingDate').set_index('meetingDate')['run_home_time'].shift(1).rolling('365D', min_periods=1).median())).reset_index()
median_win_time.rename(columns={'run_home_time': "run_home_TimeMedian"},inplace=True)

median_win_time['run_home_speedIndex'] = (median_win_time['run_home_TimeMedian'] / median_win_time['distance'])
median_win_time['run_home_speedIndex'] = MinMaxScaler().fit_transform(median_win_time[['run_home_speedIndex']])

TopazData = TopazData.merge(median_win_time, how='left', on=['track', 'distance','meetingDate'])

TopazData['run_home_TimeNorm'] = (TopazData['run_home_TimeMedian'] / TopazData['run_home_time']).clip(0.8, 1.2)
TopazData['run_home_TimeNorm'] = MinMaxScaler().fit_transform(TopazData[['runTimeNorm']])

# Sort the DataFrame by 'RaceId' and 'Box'
TopazData = TopazData.sort_values(by=['raceId', 'boxNumber'])

# Check if there is an entry equal to boxNumber + 1
TopazData['hasEntryBoxNumberPlus1'] = (TopazData.groupby('raceId')['boxNumber'].shift(1) == TopazData['boxNumber'] + 1) | (TopazData['boxNumber'] == 8)
TopazData['hasEntryBoxNumberMinus1'] = (TopazData.groupby('raceId')['boxNumber'].shift(-1) == TopazData['boxNumber'] - 1)
# Convert boolean values to 1
TopazData['hasEntryBoxNumberPlus1'] = TopazData['hasEntryBoxNumberPlus1'].astype(int)
TopazData['hasEntryBoxNumberMinus1'] = TopazData['hasEntryBoxNumberMinus1'].astype(int)
# Display the resulting DataFrame which shows adjacent Vacant Boxes
# Box 1 is treated as having a vacant box to the left always as we are looking how much space the dog has to move.
TopazData['adjacentVacantBoxes'] = 2 - TopazData['hasEntryBoxNumberPlus1'] - TopazData['hasEntryBoxNumberMinus1']
# Calculate 'hasAtLeast1VacantBox'
TopazData['hasAtLeast1VacantBox'] = (TopazData['adjacentVacantBoxes'] > 0).astype(int)

TopazData['win'] = TopazData['place'].apply(lambda x: 1 if x == 1 else 0)

grouped_data = TopazData.groupby(['track', 'distance', 'boxNumber', 'hasAtLeast1VacantBox', 'meetingDate'])['win'].mean().reset_index()
grouped_data.set_index('meetingDate', inplace=True)

# Apply rolling mean calculation to the aggregated data
box_win_percent = grouped_data.groupby(['track', 'distance', 'boxNumber', 'hasAtLeast1VacantBox']).apply(lambda x: x.sort_values('meetingDate')['win'].shift(1).rolling('365D', min_periods=1).mean()).reset_index()

# Reset index and rename columns
box_win_percent.columns = ['track', 'distance', 'boxNumber', 'hasAtLeast1VacantBox', 'meetingDate', 'rolling_box_win_percentage']

# Add to dog results dataframe
TopazData = TopazData.merge(box_win_percent, on=['track', 'distance', 'meetingDate','boxNumber','hasAtLeast1VacantBox'], how='left')

# resultMargin has the same value for 1st and 2nd placed dogs, but should be 0 for the 1st placed dog.
TopazData.loc[TopazData['place'] == 1, ['resultMargin']] = 0

TopazData['dogAge'] = (TopazData['meetingDate'] - TopazData['dateWhelped']).dt.days
scaler = MinMaxScaler()
TopazData['dogAgeScaled'] = TopazData.groupby('raceId')['dogAge'].transform(lambda x: scaler.fit_transform(x.values.reshape(-1, 1)).flatten())
TopazData['averageSpeed'] = TopazData['distance'] / TopazData['resultTime']

In [45]:
TopazData['split_time_margin'] 

0         0.0000
1         0.5495
2         0.3500
3         0.4375
4         0.4200
           ...  
984163    0.3395
984164    0.2170
984165    0.0175
984166    0.0000
984167    0.0525
Name: split_time_margin, Length: 984168, dtype: float64

In [52]:
for i,c in enumerate(TopazData.columns):
    print(i,c,TopazData[c].nunique())

0 trackCode 15
1 track 13
2 distance 34
3 raceId 134152
4 meetingDate 3385
5 raceTypeCode 64
6 raceType 64
7 runId 984168
8 dogId 36747
9 dogName 36747
10 weightInKg 247
11 incomingGrade 8
12 outgoingGrade 8
13 gradedTo 2
14 rating 65
15 raceNumber 22
16 boxNumber 8
17 boxDrawnOrder 9
18 rugNumber 10
19 startPrice 1661
20 place 8
21 unplaced 3
22 unplacedCode 3
23 scratched 1
24 prizeMoney 616
25 resultTime 32245
26 resultMargin 213
27 resultMarginLengths 213
28 startPaceCode 2
29 jumpCode 3
30 runLineCode 3
31 firstSecond 29535
32 colourCode 46
33 sex 2
34 comment 439898
35 ownerId 9029
36 trainerId 2829
37 ownerName 8946
38 ownerState 9
39 trainerName 2807
40 trainerSuburb 1351
41 trainerState 9
42 trainerDistrict 124
43 trainerPostCode 579
44 isQuad 2
45 isBestBet 2
46 damId 7163
47 damName 7156
48 sireId 586
49 sireName 586
50 dateWhelped 3880
51 totalFormCount 1
52 last5 62094
53 isLateScratching 2
54 bestTime 2252
55 bestFinishTrackAndDistance 24418
56 pir 2700
57 careerPrizeMone

In [53]:
TopazData.speedIndex

0         0.553946
1         0.553946
2         0.553946
3         0.553946
4         0.553946
            ...   
984163    0.362238
984164    0.362238
984165    0.362238
984166    0.362238
984167    0.362238
Name: speedIndex, Length: 984168, dtype: float64

In [47]:
TopazData['averageSpeed'] = TopazData['distance'] / TopazData['resultTime']
TopazData['splitMargin_1'] = np.where(TopazData['position_1'] == 1, 0, TopazData['splitMargin_1'])
TopazData['margin_from_lengths'] = pd.to_numeric(TopazData['resultMarginLengths'].str.replace('L',''))
# TopazData['win']

In [48]:
TopazData['resultMargin'] = TopazData['margin_from_lengths']

In [49]:
TopazData.shape

(984168, 96)

In [50]:
TopazData.margin_from_lengths

0          5.00
1         15.75
2         24.00
3         22.00
4          9.50
          ...  
984163     9.75
984164     7.75
984165     7.25
984166     6.25
984167     1.25
Name: margin_from_lengths, Length: 984168, dtype: float64

In [26]:
TopazData.columns

Index(['trackCode', 'track', 'distance', 'raceId', 'meetingDate',
       'raceTypeCode', 'raceType', 'runId', 'dogId', 'dogName', 'weightInKg',
       'incomingGrade', 'outgoingGrade', 'gradedTo', 'rating', 'raceNumber',
       'boxNumber', 'boxDrawnOrder', 'rugNumber', 'startPrice', 'place',
       'unplaced', 'unplacedCode', 'scratched', 'prizeMoney', 'resultTime',
       'resultMargin', 'resultMarginLengths', 'startPaceCode', 'jumpCode',
       'runLineCode', 'firstSecond', 'colourCode', 'sex', 'comment', 'ownerId',
       'trainerId', 'ownerName', 'ownerState', 'trainerName', 'trainerSuburb',
       'trainerState', 'trainerDistrict', 'trainerPostCode', 'isQuad',
       'isBestBet', 'damId', 'damName', 'sireId', 'sireName', 'dateWhelped',
       'totalFormCount', 'last5', 'isLateScratching', 'bestTime',
       'bestFinishTrackAndDistance', 'pir', 'careerPrizeMoney', 'averageSpeed',
       'time_1', 'position_1', 'splitMargin_1', 'time_2', 'position_2',
       'splitMargin_2', 'date'

In [70]:
def rolling_last(x):
    return x.iloc[-1]

In [27]:
import itertools

dataset = TopazData.copy()
print(f"dataset shape: {dataset.shape}")
dataset['meetingDate'] = pd.to_datetime(dataset['meetingDate'])

# Calculate values for dog, trainer, dam and sire
subsets = ['dog', 'trainer', 'dam', 'sire']
# subsets = ['dog']

# Use rolling window of 28, 91 and 365 days
# rolling_windows = ['28D','91D', '365D']
rolling_windows = [1,'28D','91D', '365D']
rolling_windows = [1,'365D']
# rolling_windows = [1]

# Features to use for rolling windows calculation
features = ['distance','boxNumber','runTimeNorm', 'placeLog', 'prizemoneyLog', 
            'marginLog','finishingPlaceMovement','splitMargin_1','split_runTimeNorm','run_home_TimeNorm','finishingPlaceMovement',
            'time_1','averageSpeed']

features = ['distance','boxNumber','runTimeNorm', 'place', 'resultMargin',
            'split_time_margin','split_runTimeNorm','time_1','run_home_TimeNorm','finishingPlaceMovement',
            'averageSpeed', 'win']


dam_features = ['distance','boxNumber','runTimeNorm', 'place', 'resultMargin','split_time_margin','split_runTimeNorm',
            'time_1','averageSpeed', 'win']


# Aggregation functions to apply

aggregates = ['mean']

# Keep track of generated feature names
feature_cols = []

for i in subsets:
    # Generate rolling window features
    idnumber = i + 'Id'

    subset_dataframe = dataset[['meetingDate',idnumber] + features]
    average_df = pd.DataFrame()

    for feature in features:
        # Group by 'damId' and 'meetingDate' and calculate the average of the current feature
        feature_average_df = subset_dataframe.groupby([idnumber, 'meetingDate'])[feature].mean().reset_index()
        # Rename the feature column to indicate it's the average of that feature
        feature_average_df.rename(columns={feature: f'{feature}{i}DayAverage'}, inplace=True)

        # If average_df is empty, assign the feature_average_df to it
        if average_df.empty:
            average_df = feature_average_df
        else:
            # Otherwise, merge feature_average_df with average_df
            average_df = pd.merge(average_df, feature_average_df, on=[idnumber, 'meetingDate'],how='left')

        # Assuming df is your DataFrame
    column_names = average_df.columns.tolist()
    # Columns to exclude
    columns_to_exclude = [idnumber,'meetingDate']
    # Exclude specified columns from the list
    column_names_filtered = [col for col in column_names if col not in columns_to_exclude]

    average_df.drop_duplicates(inplace=True)
    average_df['meetingDate'] = pd.to_datetime(average_df['meetingDate'])
    average_df = average_df.set_index([idnumber, 'meetingDate']).sort_index() 


    #Process Dog Stats
    for rolling_window in rolling_windows:
        print(f"dataset shape: {dataset.shape}")
        print(f'Processing {i} rolling window {rolling_window} days')

        rolling_result = (
            average_df
            .reset_index(level=0)
            .groupby(idnumber)[column_names_filtered]
            .rolling(rolling_window)  # Use timedelta for rolling window
            .agg(aggregates)
            .groupby(level=0)
            .shift(1)
        )

        # Generate list of rolling window feature names (eg: RunTime_norm_min_365D)
        agg_features_cols = [f'{i}_{f}_{a}_{rolling_window}' for f, a in itertools.product(features, aggregates)]
        # Add features to dataset
        average_df[agg_features_cols] = rolling_result
        # Keep track of generated feature names
        feature_cols.extend(agg_features_cols)
        average_df.fillna(0, inplace=True)

    

    average_df.reset_index(inplace=True)
    dataset = pd.merge(dataset,average_df,on=[idnumber, 'meetingDate'],how='left')

# Only keep data 12 months after the start date of your dataset since we've used a 365D rolling timeframe for some features
# feature_cols = np.unique(feature_cols).tolist()
# dataset = dataset[dataset['meetingDate'] >= '2021-01-01']

dataset = dataset[[
                'meetingDate',
                'state',
                'track',
                'distance',
                'raceId',
                'raceTypeCode',
                'raceNumber',
                'boxNumber',
                'rugNumber',
                'runId',
                'dogId',
                'dogName',
                'weightInKg',
                'sex',
                'trainerId',
                'trainerState',
                'damId',
                'damName',
                'sireId',
                'sireName',
                'win',
                'place',
                'resultTime',
                'resultMargin',
                'resultMarginLengths',
                'dogAgeScaled',
                'startPrice',
                # 'lastFiveWinPercentage',
                # 'lastFivePlacePercentage',
                'weightInKgScaled',
                'rolling_box_win_percentage',
                'hasEntryBoxNumberPlus1', 
                'hasEntryBoxNumberMinus1',]
                 + feature_cols
                ]

feature_cols.extend(['dogAgeScaled',
                     'boxNumber',
                # 'lastFiveWinPercentage',
                # 'lastFivePlacePercentage',
                'weightInKgScaled',
                'hasEntryBoxNumberPlus1', 'hasEntryBoxNumberMinus1',
                'rolling_box_win_percentage'])

#The below line will output your dataframe to a csv but may be too large to open in Excel.
#dataset.to_csv('testing.csv',index=False)

dataset shape: (984178, 95)
dataset shape: (984178, 95)
Processing dog rolling window 1 days
dataset shape: (984178, 95)
Processing dog rolling window 365D days
dataset shape: (984178, 131)
Processing trainer rolling window 1 days
dataset shape: (984178, 131)
Processing trainer rolling window 365D days
dataset shape: (984178, 167)
Processing dam rolling window 1 days
dataset shape: (984178, 167)
Processing dam rolling window 365D days
dataset shape: (984178, 203)
Processing sire rolling window 1 days
dataset shape: (984178, 203)
Processing sire rolling window 365D days


In [28]:
feature_cols = ['dogAgeScaled',
                     'boxNumber',
                # 'lastFiveWinPercentage',
                # 'lastFivePlacePercentage',
                'weightInKgScaled',
                'hasEntryBoxNumberPlus1', 'hasEntryBoxNumberMinus1',
                'rolling_box_win_percentage'] + [x for x in feature_cols if (('dog' in x) and ('_1' in x)) or (('dog' not in x) and ('_365' in x))]
feature_cols

['dogAgeScaled',
 'boxNumber',
 'weightInKgScaled',
 'hasEntryBoxNumberPlus1',
 'hasEntryBoxNumberMinus1',
 'rolling_box_win_percentage',
 'dog_distance_mean_1',
 'dog_boxNumber_mean_1',
 'dog_runTimeNorm_mean_1',
 'dog_place_mean_1',
 'dog_resultMargin_mean_1',
 'dog_split_time_margin_mean_1',
 'dog_split_runTimeNorm_mean_1',
 'dog_time_1_mean_1',
 'dog_run_home_TimeNorm_mean_1',
 'dog_finishingPlaceMovement_mean_1',
 'dog_averageSpeed_mean_1',
 'dog_win_mean_1',
 'dog_time_1_mean_365D',
 'trainer_distance_mean_365D',
 'trainer_boxNumber_mean_365D',
 'trainer_runTimeNorm_mean_365D',
 'trainer_place_mean_365D',
 'trainer_resultMargin_mean_365D',
 'trainer_split_time_margin_mean_365D',
 'trainer_split_runTimeNorm_mean_365D',
 'trainer_time_1_mean_365D',
 'trainer_run_home_TimeNorm_mean_365D',
 'trainer_finishingPlaceMovement_mean_365D',
 'trainer_averageSpeed_mean_365D',
 'trainer_win_mean_365D',
 'dam_distance_mean_365D',
 'dam_boxNumber_mean_365D',
 'dam_runTimeNorm_mean_365D',
 'dam_

In [29]:
feature_cols = ['dogAgeScaled',
 'boxNumber',
 'weightInKgScaled',
 'hasEntryBoxNumberPlus1',
 'hasEntryBoxNumberMinus1',
 'rolling_box_win_percentage',
 'dog_distance_mean_1',
 'dog_boxNumber_mean_1',
 'dog_runTimeNorm_mean_1',
 'dog_place_mean_1',
 'dog_resultMargin_mean_1',
 'dog_split_time_margin_mean_1',
 'dog_split_runTimeNorm_mean_1',
 'dog_time_1_mean_1',
 'dog_run_home_TimeNorm_mean_1',
 'dog_finishingPlaceMovement_mean_1',
 'dog_averageSpeed_mean_1',
 'dog_win_mean_1',
 'trainer_distance_mean_365D',
 'trainer_boxNumber_mean_365D',
 'trainer_runTimeNorm_mean_365D',
 'trainer_place_mean_365D',
 'trainer_resultMargin_mean_365D',
 'trainer_split_time_margin_mean_365D',
 'trainer_split_runTimeNorm_mean_365D',
 'trainer_time_1_mean_365D',
 'trainer_run_home_TimeNorm_mean_365D',
 'trainer_finishingPlaceMovement_mean_365D',
 'trainer_averageSpeed_mean_365D',
 'trainer_win_mean_365D',
 'dam_distance_mean_365D',
 'dam_boxNumber_mean_365D',
 'dam_runTimeNorm_mean_365D',
 'dam_place_mean_365D',
 'dam_resultMargin_mean_365D',
 'dam_split_time_margin_mean_365D',
 'dam_split_runTimeNorm_mean_365D',
 'dam_time_1_mean_365D',
 'dam_run_home_TimeNorm_mean_365D',
 'dam_finishingPlaceMovement_mean_365D',
 'dam_averageSpeed_mean_365D',
 'dam_win_mean_365D',
 'sire_distance_mean_365D',
 'sire_boxNumber_mean_365D',
 'sire_runTimeNorm_mean_365D',
 'sire_place_mean_365D',
 'sire_resultMargin_mean_365D',
 'sire_split_time_margin_mean_365D',
 'sire_split_runTimeNorm_mean_365D',
 'sire_time_1_mean_365D',
 'sire_run_home_TimeNorm_mean_365D',
 'sire_finishingPlaceMovement_mean_365D',
 'sire_averageSpeed_mean_365D',
 'sire_win_mean_365D']

In [30]:
dataset_pred = dataset.query('raceId == 1029602600')

In [34]:
dataset.dog_split_time_margin_mean_1.value_counts()

dog_split_time_margin_mean_1
0.0    984178
Name: count, dtype: int64

In [31]:
single_dog = dataset.query('dogId == 713187529')
single_dog

,meetingDate,state,track,distance,raceId,raceTypeCode,raceNumber,boxNumber,rugNumber,runId,...,sire_runTimeNorm_mean_365D,sire_place_mean_365D,sire_resultMargin_mean_365D,sire_split_time_margin_mean_365D,sire_split_runTimeNorm_mean_365D,sire_time_1_mean_365D,sire_run_home_TimeNorm_mean_365D,sire_finishingPlaceMovement_mean_365D,sire_averageSpeed_mean_365D,sire_win_mean_365D
810761,2023-11-28 00:00:00+00:00,VIC,Warragul,400,969838809,M,1,2.0,2,969839331,...,0.727978,4.082375,6.538145,0.0,0.475475,7.157572,0.727978,-0.070900,17.291530,0.195286
835010,2024-02-19 00:00:00+00:00,VIC,Geelong,400,998442400,M,1,6.0,6,998442913,...,0.729269,4.143250,6.461000,0.0,0.476491,7.193916,0.729269,0.002834,17.296653,0.187011
838181,2024-02-29 00:00:00+00:00,VIC,Shepparton,390,1001983470,M,1,6.0,6,1001983996,...,0.728620,4.159799,6.444339,0.0,0.476281,7.201918,0.728620,0.013804,17.293813,0.182185
840849,2024-03-09 00:00:00+00:00,VIC,Geelong,400,1004803600,M,1,8.0,8,1004804137,...,0.728741,4.145520,6.431988,0.0,0.476294,7.206958,0.728741,0.002276,17.292892,0.182890
842293,2024-03-14 00:00:00+00:00,VIC,Shepparton,390,1006648281,M,1,5.0,5,1006648813,...,0.728054,4.135943,6.473014,0.0,0.476106,7.221479,0.728054,0.009088,17.287805,0.181406
847716,2024-04-02 00:00:00+00:00,VIC,Geelong,400,1012516494,M,2,6.0,6,1012517022,...,0.727818,4.152908,6.511763,0.0,0.475902,7.217456,0.727818,0.014248,17.286808,0.177003
855052,2024-05-07 00:00:00+00:00,NaN,Geelong,400,1029602600,M,1,8.0,8,1029603142,...,0.729340,4.135860,6.498601,0.0,0.476403,7.197785,0.729340,0.003379,17.291804,0.174693


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assume df is your DataFrame and 'date' is your date column
df = TopazData.copy()
df['date'] = pd.to_datetime(df['date']).dt.to_period('M')

# List of relevant columns
columns = ['distance','boxNumber','runTimeNorm', 'place', 'resultMargin','splitMargin_1','split_runTimeNorm',
            'time_1','averageSpeed', 'win']  # replace with your actual column names

# Create a separate plot for each column
for col in columns:
    # Group by 'date' and calculate the percentage of missing values in each month for the column
    missing_data = df.groupby('date')[col].apply(lambda x: x.isnull().mean() * 100)

    # Plot the results
    missing_data.plot(kind='bar', figsize=(12, 6))
    plt.title(f'Percentage of missing values per month for {col}')
    plt.ylabel('Percentage of missing values')
    plt.show()

In [42]:
dataset.to_feather('topaz_data_dog.fth')

In [41]:
dataset.dogName

0             BUREKUP JACK
1           GALLOPING EMMA
2               DIEGONATOR
3                POPPY JAY
4             LEOPARDSTOWN
                ...       
2227796         HARD ON ME
2227797        IRISH NIKKI
2227798         RILEY COIN
2227799       CLASSY KAYLA
2227800    DAINTREE DAMAGE
Name: dogName, Length: 2227801, dtype: object

In [43]:
betfair_df = pd.read_feather('../data_tools/DATA/df-betfairSP.fth')

In [44]:
betfair_df['date'] =  (pd.to_datetime(betfair_df['EVENT_DT'],dayfirst=True) + pd.Timedelta(hours=7)).dt.date
betfair_df['dogName'] = betfair_df.dog.str.strip().str.upper().str.replace('.','').replace("'",'')
# betfair_df.to_csv('betfair_bsp.csv')

In [45]:
dataset['date'] = pd.to_datetime(dataset['meetingDate']).dt.date
dataset['dogName'] = dataset['dogName'].str.upper().str.replace('.','').str.replace("'",'')

In [46]:
dataset_bsp = dataset.merge(betfair_df,on=['date','dogName'],how='left')

In [47]:
def generate_prev_race(df_in, df_g, rolling_window=10, factor=''):
    df = df_in.copy()
    original_cols = df_in.columns
    df[f'prev_race'] = df_g['raceId'].shift(1).fillna('-1').astype('string')
    df[f'prev_race_date'] = df_g['date'].shift(1).fillna('-1').astype('string')
    df[f'prev_race_track'] = df_g['track'].shift(1).fillna('-1').astype('string')
    df[f'prev_race_state'] = df_g['state'].shift(1).fillna('-1').astype('string')
    df[f'next_race'] = df_g['raceId'].shift(-1).fillna('-1').astype('string')
    return(df)

In [48]:
dataset_bsp = generate_prev_race(dataset_bsp,dataset_bsp.groupby('dogId'))

In [49]:
feature_cols

['dog_distance_mean_1',
 'dog_boxNumber_mean_1',
 'dog_runTimeNorm_mean_1',
 'dog_place_mean_1',
 'dog_resultMargin_mean_1',
 'dog_split_time_margin_mean_1',
 'dog_split_runTimeNorm_mean_1',
 'dog_time_1_mean_1',
 'dog_run_home_TimeNorm_mean_1',
 'dog_finishingPlaceMovement_mean_1',
 'dog_averageSpeed_mean_1',
 'dog_win_mean_1',
 'dog_distance_mean_365D',
 'dog_boxNumber_mean_365D',
 'dog_runTimeNorm_mean_365D',
 'dog_place_mean_365D',
 'dog_resultMargin_mean_365D',
 'dog_split_time_margin_mean_365D',
 'dog_split_runTimeNorm_mean_365D',
 'dog_time_1_mean_365D',
 'dog_run_home_TimeNorm_mean_365D',
 'dog_finishingPlaceMovement_mean_365D',
 'dog_averageSpeed_mean_365D',
 'dog_win_mean_365D',
 'trainer_distance_mean_1',
 'trainer_boxNumber_mean_1',
 'trainer_runTimeNorm_mean_1',
 'trainer_place_mean_1',
 'trainer_resultMargin_mean_1',
 'trainer_split_time_margin_mean_1',
 'trainer_split_runTimeNorm_mean_1',
 'trainer_time_1_mean_1',
 'trainer_run_home_TimeNorm_mean_1',
 'trainer_finishingP

In [ ]:
feature_cols = ['dog_distance_min_1',
 'dog_boxNumber_min_1',
 'dog_runTimeNorm_min_1',
 'dog_place_min_1',
 'dog_resultMargin_min_1',
 'dog_split_time_margin_min_1',
 'dog_split_runTimeNorm_min_1',
 'dog_time_1_min_1',
 'dog_averageSpeed_min_1',
 'dog_win_min_1',
 'dogAgeScaled',
 'boxNumber',
 'weightInKgScaled',
 'hasEntryBoxNumberPlus1',
 'hasEntryBoxNumberMinus1',
 'rolling_box_win_percentage']

In [ ]:
feature_cols[0:17]

['dog_distance_min_1',
 'dog_boxNumber_min_1',
 'dog_runTimeNorm_min_1',
 'dog_place_min_1',
 'dog_resultMargin_min_1',
 'dog_split_time_margin_min_1',
 'dog_split_runTimeNorm_min_1',
 'dog_time_1_min_1',
 'dog_averageSpeed_min_1',
 'dog_win_min_1',
 'dogAgeScaled',
 'boxNumber',
 'weightInKgScaled',
 'hasEntryBoxNumberPlus1',
 'hasEntryBoxNumberMinus1',
 'rolling_box_win_percentage']

In [52]:
for i in dataset.columns:
    print(i)

meetingDate
state
track
distance
raceId
raceTypeCode
raceNumber
boxNumber
rugNumber
runId
dogId
dogName
weightInKg
sex
trainerId
trainerState
damId
damName
sireId
sireName
win
place
resultTime
resultMargin
resultMarginLengths
dogAgeScaled
startPrice
weightInKgScaled
rolling_box_win_percentage
hasEntryBoxNumberPlus1
hasEntryBoxNumberMinus1
dog_distance_mean_1
dog_boxNumber_mean_1
dog_runTimeNorm_mean_1
dog_place_mean_1
dog_resultMargin_mean_1
dog_split_time_margin_mean_1
dog_split_runTimeNorm_mean_1
dog_time_1_mean_1
dog_run_home_TimeNorm_mean_1
dog_finishingPlaceMovement_mean_1
dog_averageSpeed_mean_1
dog_win_mean_1
dog_distance_mean_365D
dog_boxNumber_mean_365D
dog_runTimeNorm_mean_365D
dog_place_mean_365D
dog_resultMargin_mean_365D
dog_split_time_margin_mean_365D
dog_split_runTimeNorm_mean_365D
dog_time_1_mean_365D
dog_run_home_TimeNorm_mean_365D
dog_finishingPlaceMovement_mean_365D
dog_averageSpeed_mean_365D
dog_win_mean_365D
trainer_distance_mean_1
trainer_boxNumber_mean_1
trainer_

In [58]:
stat_values =  pd.Series( dataset_bsp[feature_cols].fillna(-1.0).astype('float32').values.tolist())
dataset_bsp['stats_topaz'] = stat_values
dataset_bsp['dogid'] = dataset_bsp['dogId'].astype('str')
dataset_bsp['raceid'] = dataset_bsp['raceId'].astype('str')
dataset_bsp['stats_cols'] = str(feature_cols)

In [ ]:
# dataset_bsp.to_feather('topaz_data_w_bsp.fth')

In [ ]:
# dataset_bsp = pd.read_feather('topaz_data_w_bsp.fth')

In [56]:
def hash_trackname(trackname, hash_size=1024):
    hashes = [hash(x) % hash_size for x in trackname]
    return hashes

In [57]:
hashes = {x: hash(x)%1024 for x in dataset_bsp.track.unique()}

In [59]:
dataset_bsp['track_hash'] = dataset_bsp['track'].map(hashes)

In [60]:
dataset_bsp.to_feather('topaz_data_w_bsp.fth')

In [ ]:
dataset_bsp.columns

In [ ]:
dataset.query('raceId == 618363326').place

In [ ]:
len(dataset_bsp['stats_topaz'].iloc[0]

In [ ]:
df = dataset_bsp.copy()
col = 'BSP'
df = df[df['state']!='NZ']
df['date'] = pd.to_datetime(df['date']).dt.to_period('Y')
missing_data = df.groupby(['date','state'])[col].apply(lambda x: x.isnull().mean() * 100)

# Plot the results
missing_data.plot(kind='bar', figsize=(40, 6))
plt.title(f'Percentage of missing values per month for {col}')
plt.ylabel('Percentage of missing values')
plt.show()

In [35]:
dataset = pd.read_feather('../data/topaz_data_w_bsp.fth')

In [36]:
for i,c in enumerate(dataset.columns):
    print(i,c)

0 meetingDate
1 state
2 track
3 distance
4 raceId
5 raceTypeCode
6 raceNumber
7 boxNumber
8 rugNumber
9 runId
10 dogId
11 dogName
12 weightInKg
13 sex
14 trainerId
15 trainerState
16 damId
17 damName
18 sireId
19 sireName
20 win
21 place
22 resultTime
23 resultMargin
24 resultMarginLengths
25 dogAgeScaled
26 startPrice
27 weightInKgScaled
28 rolling_box_win_percentage
29 hasEntryBoxNumberPlus1
30 hasEntryBoxNumberMinus1
31 dog_distance_min_1
32 dog_boxNumber_min_1
33 dog_runTimeNorm_min_1
34 dog_place_min_1
35 dog_resultMargin_min_1
36 dog_split_time_margin_min_1
37 dog_split_runTimeNorm_min_1
38 dog_time_1_min_1
39 dog_averageSpeed_min_1
40 dog_win_min_1
41 dog_distance_min_365D
42 dog_boxNumber_min_365D
43 dog_runTimeNorm_min_365D
44 dog_place_min_365D
45 dog_resultMargin_min_365D
46 dog_split_time_margin_min_365D
47 dog_split_runTimeNorm_min_365D
48 dog_time_1_min_365D
49 dog_averageSpeed_min_365D
50 dog_win_min_365D
51 trainer_distance_min_1
52 trainer_boxNumber_min_1
53 trainer_ru

In [40]:
dataset.dog_split_runTimeNorm_min_1.value_counts()

dog_split_runTimeNorm_min_1
0.000000    694670
0.500000     13648
1.000000      2018
0.503922       692
0.500000       628
             ...  
0.446961         1
0.450014         1
0.436149         1
0.421204         1
0.476906         1
Name: count, Length: 375162, dtype: int64

In [ ]:
dataset.to_csv('../data/topaz_data_w_bsp.csv')

In [ ]:
dataset.shape

In [ ]:
feature_cols

In [ ]:
for col in dataset.columns:
    print(col)

In [ ]:
feature_cols

In [ ]:
dataset.raceId.nunique()


In [61]:
# Your existing function to generate date range
def generate_date_range(start_date, end_date):
    start_date = start_date
    end_date = end_date

    date_list = []
    current_date = start_date
    while current_date <= end_date:
        date_list.append(current_date.strftime("%Y-%m-%d"))
        current_date += timedelta(days=31)

    return date_list

# Example usage:
start_date = datetime(2024,1,1)
end_date = (datetime.today() + timedelta(days=31))

# Generate the date range
date_range = generate_date_range(start_date, end_date)

In [62]:


def topaz_races_threaded(buckets, topaz_api, progress):
    all_races = []
    # print(f"{buckets=}")
    errors = []
    for bucket in buckets:
        start_date, end_date, state = bucket
        # print(bucket)
        try:
            races = topaz_api.get_races(from_date=start_date, to_date=end_date, owning_authority_code=state)
            races['state'] = state
            all_races.append(races)
        except requests.HTTPError as http_err:
            print(f'HTTP error occurred: {http_err}')
            errors.append(bucket)
            pass
        # time.sleep(2)
        progress.update()
    return all_races,errors

def get_topaz_races(start_date, end_date, states, topaz_api:TopazAPI):
    date_range = generate_date_range(start_date, end_date)
    starts = date_range[:-1]
    ends = date_range[1:]
    date_range_states = [(start, end, state) for start, end in zip(starts, ends) for state in states]

    print(f"Created {len(date_range_states)} date ranges for {len(states)} states")

    num_workers = min(6, len(date_range_states))  # Adjust this value based on your system's capabilities
    chunk_size = math.ceil(len(date_range_states) / num_workers)

    chunks = [date_range_states[i:i + chunk_size] for i in range(0, len(date_range_states), chunk_size)]
    
    print(chunks)
    print(len(chunks))
    _process_jobs = []
    bars = []
    results = []
    errors = []
    for i in range(num_workers):
        bars.append(tqdm(total=len(chunks[i]), position=i)) 
        # time.sleep(2)
    with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:


        for i,chunk in enumerate(chunks):
            _process_jobs.append(executor.submit(topaz_races_threaded, chunk, topaz_api, bars[i]))

        # results = []
        for job in concurrent.futures.as_completed(_process_jobs):
            result,error = job.result()
            errors.extend(error)
            results.append(result)

    

    # results = []
    print(errors)
    return results

In [66]:
start_date = datetime(2015,1,1)
end_date = (datetime.today() + timedelta(days=31))
states = states = ['NSW', 'VIC', 'NZ', 'QLD', 'SA', 'WA', 'TAS', 'NT', 'ACT']
states = ['NZ']
states = ['SA']

In [67]:
output = get_topaz_races(start_date, end_date, states, topaz_api)
output_flat = [item for sublist in output for item in sublist]
all_races_df = pd.concat(output_flat,ignore_index=True).reset_index(drop=True)
all_races_df.to_csv('all_races_topas_SA.csv',index=False)

Created 111 date ranges for 1 states
[[('2015-01-01', '2015-02-01', 'SA'), ('2015-02-01', '2015-03-04', 'SA'), ('2015-03-04', '2015-04-04', 'SA'), ('2015-04-04', '2015-05-05', 'SA'), ('2015-05-05', '2015-06-05', 'SA'), ('2015-06-05', '2015-07-06', 'SA'), ('2015-07-06', '2015-08-06', 'SA'), ('2015-08-06', '2015-09-06', 'SA'), ('2015-09-06', '2015-10-07', 'SA'), ('2015-10-07', '2015-11-07', 'SA'), ('2015-11-07', '2015-12-08', 'SA'), ('2015-12-08', '2016-01-08', 'SA'), ('2016-01-08', '2016-02-08', 'SA'), ('2016-02-08', '2016-03-10', 'SA'), ('2016-03-10', '2016-04-10', 'SA'), ('2016-04-10', '2016-05-11', 'SA'), ('2016-05-11', '2016-06-11', 'SA'), ('2016-06-11', '2016-07-12', 'SA'), ('2016-07-12', '2016-08-12', 'SA')], [('2016-08-12', '2016-09-12', 'SA'), ('2016-09-12', '2016-10-13', 'SA'), ('2016-10-13', '2016-11-13', 'SA'), ('2016-11-13', '2016-12-14', 'SA'), ('2016-12-14', '2017-01-14', 'SA'), ('2017-01-14', '2017-02-14', 'SA'), ('2017-02-14', '2017-03-17', 'SA'), ('2017-03-17', '2017-04

  0%|          | 0/19 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

[]


C:\Users\Nick\AppData\Local\Temp\ipykernel_8880\1557507502.py:3: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_races_df = pd.concat(output_flat,ignore_index=True).reset_index(drop=True)


In [ ]:
import torch
from torch import nn

# Assume you have 1000 unique track names and you want to create an embedding of size 50 for each track name
num_tracknames = 1000
embedding_dim = 50

# Create an embedding layer
embedding = nn.Embedding(num_tracknames, embedding_dim)

# Assume trackname_indices is a tensor of integers, where each integer is the index of a track name in the dictionary
# For example, you can create it by replacing each track name in your data with its index in the dictionary of track names
trackname_indices = torch.tensor([0, 1, 2, 3, 4])  # replace with your actual data

# Get the embeddings of the track names
trackname_embeddings = embedding(trackname_indices)

print(trackname_embeddings)

In [ ]:
all_races_df['date'] = pd.to_datetime(all_races_df['raceStart']).dt.date

In [ ]:
meeting_ids = list(all_races_df['meetingId'].unique())

In [ ]:
def topaz_meeting_runs_threaded(chunk,topaz_api:TopazAPI,progress):
    race_runs = []
    race_results = []
    errors = []
    for race_id in chunk:
        try:
            # race_run = topaz_api.get_race_runs(race_id=race_id)
            # race_runs.append(race_run)
            time.sleep(0.3)
            race_result_json = topaz_api.get_race_result(race_id = race_id)
            try:
                race_run.to_feather(f"race_runs/{race_id}_run.fth")
                race_result_df = pd.DataFrame.from_dict([race_result_json])
                race_result_df.to_feather(f"results/{race_id}_results.fth")
            except Exception as e:
                print(e)

        except requests.HTTPError as http_err:
            print(f'HTTP error occurred: {http_err}')
            if http_err.response.status_code == 429:

                time.sleep(120)
            errors.append(race_id)
            pass
        progress.update()

    return race_runs,race_results,errors

def topaz_meeting_run_getter(race_id_list,topaz_api:TopazAPI):

    print(f"Fetching data for  {len(race_id_list)}")

    num_workers = 6
    chunk_size = math.ceil(len(race_id_list) / num_workers)

    chunks = [race_id_list[i:i + chunk_size] for i in range(0, len(race_id_list), chunk_size)]
    
    print(chunks)
    print(len(chunks))
    _process_jobs = []
    bars = []
    race_runs = []
    results = []
    errors = []
    for i in range(num_workers):
        bars.append(tqdm(total=len(chunks[i]), position=i)) 
        # time.sleep(2)
    with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:


        for i,chunk in enumerate(chunks):
            _process_jobs.append(executor.submit(topaz_meeting_runs_threaded, chunk, topaz_api, bars[i]))

        # results = []
        for job in concurrent.futures.as_completed(_process_jobs):
            race_run,result_json,error = job.result()
            race_runs.extend(race_run)
            errors.extend(error)
            results.extend(result_json)

    

    # results = []
    print(errors)
    return race_runs,results,errors

In [ ]:
race_ids = list(all_races_df['raceId'].unique())

In [ ]:
race_id = 837931333

In [ ]:
race_run = topaz_api.get_race_runs(race_id=race_id)
race_result_json,response = topaz_api.get_race_result(race_id=race_id)

In [ ]:
rate_lim_left = int(response.headers['ratelimit-remaining'])
reset_time = int(response.headers['ratelimit-reset'])

In [ ]:
reset_time

In [ ]:
race_run_from_json = pd.DataFrame(race_result_json['runs'])

In [ ]:
race_run_from_json 

In [ ]:
split_times = pd.DataFrame(race_result_json['splitTimes'])
split_times_1 = split_times[split_times['splitTimeMarker'] == 1][['runId','time','position','splitMargin']]
split_time_2 = split_times[split_times['splitTimeMarker'] == 2][['runId','time','position','splitMargin']]
split_times = split_times_1.merge(split_time_2, on='runId',suffixes=('_1','_2'),how='left')

In [ ]:
split_times

In [ ]:
race_run_from_json 

In [ ]:
race_result_json

In [ ]:
race_result_json_meeting = topaz_api.get_meeting_details(meeting_id = 809592457)

In [ ]:
race_result_df

In [ ]:
def topaz_race_runs_threaded(chunk,topaz_api:TopazAPI,progress):
    race_runs = []
    race_results = []
    errors = []
    for race_id in chunk:
        try:
            race_run = topaz_api.get_race_runs(race_id=race_id)
            race_runs.append(race_run)
            time.sleep(0.3)
            race_result_json = topaz_api.get_race_result(race_id = race_id)
            try:
                race_run.to_feather(f"race_runs/{race_id}_run.fth")
                race_result_df = pd.DataFrame.from_dict([race_result_json])
                race_result_df.to_feather(f"results/{race_id}_results.fth")
            except Exception as e:
                print(e)

        except requests.HTTPError as http_err:
            print(f'HTTP error occurred: {http_err}')
            if http_err.response.status_code == 429:

                time.sleep(120)
            errors.append(race_id)
            pass
        progress.update()

    return race_runs,race_results,errors

In [ ]:
def topaz_race_run_getter(race_id_list,topaz_api:TopazAPI):

    print(f"Fetching data for  {len(race_id_list)}")

    num_workers = 6
    chunk_size = math.ceil(len(race_id_list) / num_workers)

    chunks = [race_id_list[i:i + chunk_size] for i in range(0, len(race_id_list), chunk_size)]
    
    print(chunks)
    print(len(chunks))
    _process_jobs = []
    bars = []
    race_runs = []
    results = []
    errors = []
    for i in range(num_workers):
        bars.append(tqdm(total=len(chunks[i]), position=i)) 
        # time.sleep(2)
    with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:


        for i,chunk in enumerate(chunks):
            _process_jobs.append(executor.submit(topaz_race_runs_threaded, chunk, topaz_api, bars[i]))

        # results = []
        for job in concurrent.futures.as_completed(_process_jobs):
            race_run,result_json,error = job.result()
            race_runs.extend(race_run)
            errors.extend(error)
            results.extend(result_json)

    

    # results = []
    print(errors)
    return race_runs,results,errors

In [ ]:
all_races_df = pd.read_csv('all_races_topas.csv', header=0)
i = 0
race_ids = list(all_races_df['raceId'].unique())
# subset_ids = race_ids[i:min(len(race_ids),i+100)]
# race_runs,results,errors = topaz_race_run_getter(subset_ids,topaz_api)

In [ ]:
all_races_df

In [ ]:
test_race_run_df = pd.read_feather('race_runs/837931333_run.fth')

In [ ]:
test_race_results_json_df = pd.read_feather('results/837931333_results.fth')

In [ ]:
test_race_results_json_df

In [ ]:
for i in range(0,len(race_ids),100):
    subset_ids = race_ids[i:min(len(race_ids),i+1000)]
    race_runs,results,errors = topaz_race_run_getter(subset_ids,topaz_api)

    results_df = pd.DataFrame.from_dict(results)
    all_race_runs = pd.concat(race_runs,ignore_index=True).reset_index(drop=True)
    all_race_runs.to_feather(f'race_runs/{i}_topaz_race_runs.fth')
    results_df.to_feather(f"results/{i}_topaz_results.fth")
    # with 

In [ ]:
results

In [ ]:
race_ids = list(all_races_df['raceId'].unique())
code = "NSW-VIC"
for race_id in tqdm(race_ids, desc="Processing races", unit="race"):
    result_retries = 10

    while result_retries > 0:
        # Use tqdm to create a progress bar
        # Get race run data
        try:
            race_run = topaz_api.get_race_runs(race_id=race_id)
            race_result_json = topaz_api.get_race_result(race_id=race_id)
            file_path = code + '_DATA.csv'
            file_exists = os.path.isfile(file_path)
            header_param = not file_exists

            race_result = pd.DataFrame.from_dict([race_result_json])
            split_times_df = pd.DataFrame(race_result['splitTimes'].tolist(),index=race_result.index)

            splits_dict = split_times_df.T.stack().to_frame()
            splits_dict.reset_index(drop=True, inplace= True)
            splits_normalised = pd.json_normalize(splits_dict[0])

            if len(splits_normalised) == 0:
                race_run.to_csv(code + '_DATA.csv', mode='a', header=header_param, index=False)
                break

            first_split = splits_normalised[splits_normalised['splitTimeMarker'] == 1]
            first_split = first_split[['runId','position','time']]
            first_split = first_split.rename(columns={'position':'firstSplitPosition','time':'firstSplitTime'})
            second_split = splits_normalised[splits_normalised['splitTimeMarker'] == 2]
            second_split = second_split[['runId','position','time']]
            second_split = second_split.rename(columns={'position':'secondSplitPosition','time':'secondSplitTime'})

            split_times = splits_normalised[['runId']]
            split_times = pd.merge(split_times,first_split,how='left',on=['runId'])
            split_times = pd.merge(split_times,second_split,how='left',on=['runId'])

            race_run = pd.merge(race_run,split_times,how='left',on=['runId'])
            race_run.drop_duplicates(inplace=True)
            race_run.to_csv(code + '_DATA.csv', mode='a', header=header_param, index=False)
            break
        except requests.HTTPError as http_err:
            if http_err.response.status_code == 404:
                file_path = code + '_DATA.csv'
                file_exists = os.path.isfile(file_path)
                header_param = not file_exists
                race_run.to_csv(code + '_DATA.csv', mode='a', header=header_param, index=False)
                break
        except Exception as e:
            print(race_id)
            result_retries -= 1
            if result_retries > 0:
                time.sleep(15)
            else:
                time.sleep(120)

In [ ]:
# Iterate over 7-day blocks
for i in range(0, len(date_range), 10):
    start_block_date = date_range[i]
    print(start_block_date)
    end_block_date = date_range[min(i + 9, len(date_range) - 1)]  # Ensure the end date is within the range

    codes = ['NT','VIC','NSW','SA','WA','QLD','TAS','NZ']
    codes = ['VIC', "NSW"]
    all_races = []
    for code in codes:
        
        print(code)
        retries = 10  # Number of retries
        while retries > 0:
            try:
                races = topaz_api.get_races(from_date=start_block_date, to_date=end_block_date, owning_authority_code=code)
                all_races.append(races)
                break  # Break out of the loop if successful
            except requests.HTTPError as http_err:
                if http_err.response.status_code == 429:
                    retries -= 1
                    if retries > 0:
                        print(f"Rate limited. Retrying in 121 seconds...")
                        time.sleep(121)
                    else:
                        print("Max retries reached. Moving to the next block.")
                else:
                    print(f"Error fetching races for {code}: {http_err.response.status_code}")
                    retries -= 1
                    if retries > 0:
                        print(f"Retrying in 30 seconds...")
                        time.sleep(30)
                    else:
                        print("Max retries reached. Moving to the next block.")

    try:
        all_races_df = pd.concat(all_races,ignore_index=True).reset_index(drop=True)
    except ValueError:
        continue

    # Extract unique race IDs
    race_ids = list(all_races_df['raceId'].unique())

    for race_id in tqdm(race_ids, desc="Processing races", unit="race"):
        result_retries = 10

        while result_retries > 0:
            # Use tqdm to create a progress bar
            # Get race run data
            try:
                race_run = topaz_api.get_race_runs(race_id=race_id)
                race_result_json = topaz_api.get_race_result(race_id=race_id)
                file_path = code + '_DATA.csv'
                file_exists = os.path.isfile(file_path)
                header_param = not file_exists

                race_result = pd.DataFrame.from_dict([race_result_json])
                split_times_df = pd.DataFrame(race_result['splitTimes'].tolist(),index=race_result.index)

                splits_dict = split_times_df.T.stack().to_frame()
                splits_dict.reset_index(drop=True, inplace= True)
                splits_normalised = pd.json_normalize(splits_dict[0])

                if len(splits_normalised) == 0:
                    race_run.to_csv(code + '_DATA.csv', mode='a', header=header_param, index=False)
                    break

                first_split = splits_normalised[splits_normalised['splitTimeMarker'] == 1]
                first_split = first_split[['runId','position','time']]
                first_split = first_split.rename(columns={'position':'firstSplitPosition','time':'firstSplitTime'})
                second_split = splits_normalised[splits_normalised['splitTimeMarker'] == 2]
                second_split = second_split[['runId','position','time']]
                second_split = second_split.rename(columns={'position':'secondSplitPosition','time':'secondSplitTime'})

                split_times = splits_normalised[['runId']]
                split_times = pd.merge(split_times,first_split,how='left',on=['runId'])
                split_times = pd.merge(split_times,second_split,how='left',on=['runId'])

                race_run = pd.merge(race_run,split_times,how='left',on=['runId'])
                race_run.drop_duplicates(inplace=True)
                race_run.to_csv(code + '_DATA.csv', mode='a', header=header_param, index=False)
                break
            except requests.HTTPError as http_err:
                if http_err.response.status_code == 404:
                    file_path = code + '_DATA.csv'
                    file_exists = os.path.isfile(file_path)
                    header_param = not file_exists
                    race_run.to_csv(code + '_DATA.csv', mode='a', header=header_param, index=False)
                    break
            except Exception as e:
                print(race_id)
                result_retries -= 1
                if result_retries > 0:
                    time.sleep(15)
                else:
                    time.sleep(120)